In [1]:
%run ./pulsar_v1.0.ipynb

Epoch 1: loss=0.154, accuracy=0.959/0.972]
Epoch 2: loss=0.131, accuracy=0.966/0.972]
Epoch 3: loss=0.136, accuracy=0.967/0.970]
Epoch 4: loss=0.133, accuracy=0.968/0.970]
Epoch 5: loss=0.121, accuracy=0.968/0.969]
Epoch 6: loss=0.145, accuracy=0.968/0.974]
Epoch 7: loss=0.122, accuracy=0.970/0.975]
Epoch 8: loss=0.127, accuracy=0.970/0.976]
Epoch 9: loss=0.125, accuracy=0.970/0.976]
Epoch 10: loss=0.134, accuracy=0.968/0.976]

Final Test: final accuracy = 0.976


In [2]:
def pulsar_exec(epoch_count=10, mb_size=10, report=1, adjust_ratio=False):
    load_pulsar_dataset(adjust_ratio)
    init_model()
    train_and_test(epoch_count, mb_size, report)

In [3]:
def load_pulsar_dataset(adjust_ratio):
    pulsars, stars = [], []
    with open('../../DataSets/pulsar_stars.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        for row in csvreader:
            if row[8] == '1': pulsars.append(row)
            else: stars.append(row)

    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8, 1

    star_cnt, pulsar_cnt = len(stars), len(pulsars)

    if adjust_ratio:
        data = np.zeros([2*star_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
        for n in range(star_cnt):
            data[star_cnt + n] = np.asarray(pulsars[n % pulsar_cnt], dtype='float32')
    else:
        data = np.zeros([star_cnt + pulsar_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
        data[star_cnt:, :] = np.asarray(pulsars, dtype='float32')

In [4]:
def eval_accuracy(output, y):
    est_yes = np.greater(output, 0)
    ans_yes = np.greater(y, 0.5)
    est_no = np.logical_not(est_yes)
    ans_no = np.logical_not(ans_yes)

    tp = np.sum(np.logical_and(est_yes, ans_yes))
    fp = np.sum(np.logical_and(est_yes, ans_no))
    fn = np.sum(np.logical_and(est_no, ans_yes))
    tn = np.sum(np.logical_and(est_no, ans_no))

    accuracy = safe_div(tp+tn, tp+tn+fp+fn)
    precision = safe_div(tp, tp+fp)
    recall = safe_div(tp, tp+tn)
    f1 = 2 * safe_div(recall*precision, recall+precision)

    return [accuracy, precision, recall, f1]

def safe_div(p, q):
    p, q = float(p), float(q)
    if np.abs(q) < 1.0e-20: return np.sign(p)
    return p / q


In [5]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()

    for epoch in range(epoch_count):
        losses = []

        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, _ = run_train(train_x, train_y)
            losses.append(loss)

        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            acc_str = ','.join(['%5.3f']*4) % tuple(acc)
            print('Epoch {}: loss={:5.3f}, result={}'.format(epoch+1, np.mean(losses), acc_str))

    acc = run_test(test_x, test_y)
    acc_str = ','.join(['%5.3f']*4) % tuple(acc)
    print('\nFinal Test: final result = {}'.format(acc_str))


In [6]:
pulsar_exec(adjust_ratio=True)


Epoch 1: loss=0.405, result=0.923,0.939,0.902,0.920
Epoch 2: loss=0.376, result=0.918,0.983,0.848,0.910
Epoch 3: loss=0.377, result=0.904,0.879,0.934,0.906
Epoch 4: loss=0.364, result=0.915,0.987,0.839,0.907
Epoch 5: loss=0.378, result=0.925,0.941,0.905,0.923
Epoch 6: loss=0.367, result=0.923,0.982,0.859,0.916
Epoch 7: loss=0.369, result=0.917,0.986,0.844,0.909
Epoch 8: loss=0.363, result=0.927,0.940,0.910,0.925
Epoch 9: loss=0.349, result=0.926,0.981,0.867,0.920
Epoch 10: loss=0.378, result=0.915,0.909,0.919,0.914

Final Test: final result = 0.915,0.909,0.919,0.914
